# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 2029, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 2029 (delta 248), reused 323 (delta 188), pack-reused 1618 (from 1)
Receiving objects: 100% (2029/2029), 141.68 MiB | 31.20 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.
Updating files: 100% (238/238), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.0 MB/s eta 0:00:00


In [4]:
! cd /kaggle/working/RECsys_Challenge2024 && python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function '__pyx_pf_32MatrixFactorization

In [5]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os

np.random.seed(42)

## Import the repository

In [6]:
from github import Github, Auth

# Authenticate using a personal access token
auth_token = Auth.Token(token)
github_client = Github(auth=auth_token)

# Define the repository name you want to find
target_repo_name = 'RECsys_Challenge2024'
repo = None

# Search for the repository in the user's repositories
try:
    for repository in github_client.get_user().get_repos():
        if repository.name == target_repo_name:
            repo = repository
            print(f"Repository '{target_repo_name}' found.")
            break
    if repo is None:
        print(f"Repository '{target_repo_name}' not found.")
except Exception as e:
    print("An error occurred while accessing the repositories:", e)

Repository 'RECsys_Challenge2024' found.


In [7]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    """
    Uploads a file from Kaggle to GitHub, updating it if it already exists in the repository,
    or creating it if it does not.

    Parameters:
    - filepath_kaggle: Path to the file in the Kaggle environment.
    - filepath_github: Target path in the GitHub repository where the file should be uploaded.
    - commit_message: Message for the commit on GitHub.
    """
    try:
        
        # Check if the file already exists in the GitHub repository
        contents = repo.get_contents(filepath_github)
        
        # If it exists, update the file
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(
                contents.path, commit_message, file.read(), contents.sha
            )
        print(f"File '{filepath_github}' updated successfully.")
    
    except Exception as e:
        
        # If the file does not exist, create it
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(
                filepath_github, commit_message, file.read()
            )
        print(f"File '{filepath_github}' created successfully.")

In [8]:
config = {
    'model': 'RP3beta',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_RP3beta.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [9]:
try:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/' \
        f'{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [10]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [11]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [12]:
%cd /kaggle/working/RECsys_Challenge2024/

/kaggle/working/RECsys_Challenge2024


In [13]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [14]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

def objective_function_RP3beta(optuna_trial):
    
    recommender_instance = RP3betaRecommender(URM_train)
    
    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 3, log=False),
                   "beta": optuna_trial.suggest_float("beta", 0, 3, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [15]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_RP3beta, n_trials=100)

[I 2024-11-19 00:45:19,178] Using an existing study with name 'hyperparameters_tuning_RP3beta' instead of creating a new one.


RP3betaRecommender: Similarity column 38121 (100.0%), 2072.87 column/sec. Elapsed time 18.39 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 33.29 sec. Users per second: 1069


[I 2024-11-19 00:46:13,044] Trial 100 finished with value: 0.05206425684006616 and parameters: {'topK': 29, 'alpha': 0.5782164575412156, 'beta': 0.42167415231201083, 'normalize_similarity': True, 'implicit': False}. Best is trial 89 with value: 0.052120791220452205.


RP3betaRecommender: Similarity column 38121 (100.0%), 2034.94 column/sec. Elapsed time 18.73 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 33.44 sec. Users per second: 1064


[I 2024-11-19 00:47:07,416] Trial 101 finished with value: 0.05203780850261918 and parameters: {'topK': 35, 'alpha': 0.3650604020313712, 'beta': 0.41448476856304284, 'normalize_similarity': True, 'implicit': False}. Best is trial 89 with value: 0.052120791220452205.


...

[I 2024-11-19 02:28:16,569] Trial 197 finished with value: 0.04766777258264647 and parameters: {'topK': 88, 'alpha': 0.0062757234876637535, 'beta': 0.12044005428727636, 'normalize_similarity': True, 'implicit': False}. Best is trial 157 with value: 0.05401447501320864.


RP3betaRecommender: Similarity column 38121 (100.0%), 1886.35 column/sec. Elapsed time 20.21 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 34.07 sec. Users per second: 1045


[I 2024-11-19 02:29:13,331] Trial 198 finished with value: 0.03597879250028289 and parameters: {'topK': 59, 'alpha': 0.1824370277220093, 'beta': 0.048446502913955525, 'normalize_similarity': False, 'implicit': False}. Best is trial 157 with value: 0.05401447501320864.


RP3betaRecommender: Similarity column 38121 (100.0%), 2093.15 column/sec. Elapsed time 18.21 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 32.38 sec. Users per second: 1099


[I 2024-11-19 02:30:05,626] Trial 199 finished with value: 0.05283782611700672 and parameters: {'topK': 26, 'alpha': 0.12719710168567624, 'beta': 0.17130310947902289, 'normalize_similarity': True, 'implicit': False}. Best is trial 157 with value: 0.05401447501320864.


## Some optuna visualizations on recommender parameters

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [17]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [18]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'/kaggle/working/TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = RP3betaRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

RP3betaRecommender: Similarity column 38121 (100.0%), 1874.89 column/sec. Elapsed time 20.33 sec


# Testing

Create the recommendations for the submission. 

In [19]:
def create_submission(data_target_users_test, recommender_instance, cutoff=10, output_file=f"/kaggle/working/submission_{config['model']}.csv"):
    
    target_result = []

    for target in data_target_users_test["user_id"]:
        target_result.append(recommender_instance.recommend(target, cutoff=cutoff, remove_seen_flag=True))

    user_ids = data_target_users_test["user_id"]
    formatted_data = {
        "user_id": user_ids,
        "item_list": [" ".join(map(str, items)) for items in target_result]
    }

    submission_df = pd.DataFrame(formatted_data)
    submission_df.to_csv(output_file, index=False, header=["user_id", "item_list"])

    print(f"Submission file saved as {output_file}")

In [20]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance)

Submission file saved as /kaggle/working/submission_RP3beta.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [21]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/RP3betaRecommender/best_params_RP3beta.json' updated successfully.


Save the history of the tuned model.

In [22]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'TrainedModels/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)'
    )

File 'TrainedModels/RP3betaRecommender/history_RP3beta.db' updated successfully.


Save the best trained model and its submission.

In [ ]:
if config['save_github'] and config['tune_best_params']:
    
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'TrainedModels/{config["model"]}Recommender/Submission/'\
                    f'submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)'
            )
    
    recommender_instance.save_model(folder_path='/kaggle/working/', file_name=f'best_{config["model"]}_tuned')
    
    zip_file_path = f'/kaggle/working/best_{config["model"]}_tuned.zip'
    
    # 50MB limitation management for GitHub pushes. 
    try:
        upload_file(
            zip_file_path,  
            f'TrainedModels/{config["model"]}Recommender/best_{config["model"]}_tuned.zip', 
            f'New {config["model"]} recommender tuned with best parameters (from kaggle notebook)'
        )
    except Exception as e:
        print(f"\nThe best recommender was not saved on GitHub because its size is more than 50 MB.")

File 'TrainedModels/RP3betaRecommender/Submission/submission_RP3beta.csv' updated successfully.
RP3betaRecommender: Saving model in file '/kaggle/working/best_RP3beta_tuned'
RP3betaRecommender: Saving complete
File 'TrainedModels/RP3betaRecommender/best_RP3beta_tuned.zip' updated successfully.
